In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import optim

import numpy as np

In [ ]:
# UNIPoint model
#-------------------------------------------------------
# dimensions be like:
#-------------------------------------------------------
# input -> hidden state -> parameters -> basis functions -> intensity function (output)
#-------------------------------------------------------
# (batch_size, seq_len, n_features) -> [for iteration we take input] (batch_size, 1, n_features) ->
# -> (batch_size, 1, 1) -> (batch_size, 1, n_parameters * n_basis_functions) -> 
# -> (batch_size, 1, n_basis_functions) -> (batch_size, 1, 1) 
#-------------------------------------------------------


class UNIPoint(nn.Module):
    def __init__(self, batch_size, seq_len, n_features, n_parameters, n_basis_functions):
      """
      Input parameters:
      n_neurons - number of neurons inside RNN
      n_parameters - expecteed number of parameters in basis function
      n_basis_functions - number of basis functions
      """
      super(UNIPoint, self).__init__()

      self.rnn = nn.RNNCell(n_features, 1)
      self.hx = torch.randn(batch_size, 1) # initialize hidden state 
      self.h2p = nn.Linear(1, n_parameters * n_basis_functions)
      self.basis_res = torch.randn(batch_size, n_basis_functions) #initialize matrix for basis f-s calculations results
      self.Softplus = torch.nn.Softplus(beta = 1)

      self.seq_len = seq_len
      self.n_basis_functions = n_basis_functions

    def ReLU(self, parameter_1, parameter_2, time):
      """Function to apply Rectified Linear Unit (ReLU) as basis function inside network 
        Input parameters:
          parameters - alpha, beta for basis function's value calculation
          time - column-vector with time which had been spent since the begining of 
                  temporal point process (TPP)
      """
      self.output = torch.relu(self.parameters[:,parameter_1] * time + self.parameters[:,parameter_2] ) 
      return self.output
    
    def PowerLaw(self, parameter_1, parameter_2, time): # need to fix (see ReLU parameters and do the same)
      """Function to apply Power Law (PL) as basis function inside network 
        Input parameters:
          parameters - alpha, beta for basis function's value calculation
          time - column-vector with time which had been spent since the begining of 
                  temporal point process (TPP)
      """
      self.output = self.parameters[:,parameter_1] * (1 + time)**( - self.parameters[:,parameter_2])
      return self.output


    def forward(self, X):
      """Input parameters:
          X - batch with data 
          time - column-vector with interarrival time in temporal point process (TPP)
      """
        
      hidden_states, intensity_values = [], []
      
      # for each time step (here X shape is (batch_size, seq_len, n_features) )
      for i in range(self.seq_len):

          self.hx = self.rnn(X[:,i,:], self.hx)
          self.parameters = self.h2p(self.hx)
          
          for function in range(self.n_basis_functions): 
              # calculating numbers of parameters to take for basis function
              par1 = 2 * function
              par2 = 2 * function + 1
              self.basis_res[:, function] = self.ReLU(par1, par2, X[:,i,1]) # here X[:,i,1] - tau
          
          self.sum_res = torch.sum(self.basis_res, 1)

          self.intensity_res = self.Softplus(self.sum_res)

          hidden_states.append(self.hx)
          intensity_values.append(self.intensity_res)
          
          print("Sequence ", i+1, "out of", X.shape[1])
          
      return hidden_states, intensity_values


In [ ]:
# model evaluation

# X_batch dimension = (batch_size, seq_len, n_features)
X_batch = torch.tensor([[[1, 0.1],
                         [0, 0.2],
                         [0, 0.3],
                         [1, 0.4],
                         [0, 0.5],
                         [0, 0.6],
                         [1, 0.7]],
                        [[1, 1.0],
                         [0, 0.8], 
                         [0, 0.6],
                         [0, 0.4],
                         [0, 0.2],
                         [1, 0.1],
                         [0, 0.2]]], dtype = torch.float)

FIXED_BATCH_SIZE = 2 # our batch size is fixed for now
SEQ_LEN = 7
N_FEATURES = 2

N_PARAMETERS = 2
N_BASIS_FUNCTIONS = 4


model = UNIPoint(FIXED_BATCH_SIZE, SEQ_LEN, N_FEATURES, N_PARAMETERS, N_BASIS_FUNCTIONS)
print(model)
hidden_states, intensity_values = model(X_batch)
print()
print('hidden_states')
print(hidden_states) # contains all output for all timesteps
print()
print('intensity_values')
print(intensity_values)


UNIPoint(
  (rnn): RNNCell(2, 1)
  (h2p): Linear(in_features=1, out_features=8, bias=True)
  (Softplus): Softplus(beta=1, threshold=20)
)
Sequence  1 out of 7
Sequence  2 out of 7
Sequence  3 out of 7
Sequence  4 out of 7
Sequence  5 out of 7
Sequence  6 out of 7
Sequence  7 out of 7

hidden_states
[tensor([[-0.7855],
        [-0.8579]], grad_fn=<TanhBackward>), tensor([[0.5136],
        [0.1098]], grad_fn=<TanhBackward>), tensor([[-0.5213],
        [-0.4676]], grad_fn=<TanhBackward>), tensor([[-0.5383],
        [ 0.1353]], grad_fn=<TanhBackward>), tensor([[ 0.1073],
        [-0.1815]], grad_fn=<TanhBackward>), tensor([[-0.4660],
        [-0.5516]], grad_fn=<TanhBackward>), tensor([[-0.7187],
        [ 0.3599]], grad_fn=<TanhBackward>)]

intensity_values
[tensor([2.7563, 2.6279], grad_fn=<SoftplusBackward>), tensor([1.3954, 1.9241], grad_fn=<SoftplusBackward>), tensor([2.3411, 2.3220], grad_fn=<SoftplusBackward>), tensor([2.3647, 1.7799], grad_fn=<SoftplusBackward>), tensor([1.8335, 2.